## Implementación de 1-n NQK con D=9 sin PCA Original Scheme

### I) Data re-uploading QNN (de 1 qubit, 9 features, 6 capas)

Datos de prueba

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import matplotlib.pyplot as plt

#Semilla
np.random.seed(42)

# Función para calcular la matriz de densidad
def density_matrix(state):
    """Calcula la matriz de densidad a partir de un vector de estado  

    Args:
        state (array[complex]): arreglo que representa el vector de estado
    Returns:
        dm: (array[complex]): arreglo que representa la matriz de densidad
    """
    return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = np.array([label_0, label_1], requires_grad=False)

Circuito del QNN con data reloading y fidelity

In [2]:
dev = qml.device("lightning.qubit", wires=1)


@qml.qnode(dev)
def qcircuit(params, x, y):
    """Circuito variacional con data reloading.

    Args:
        params (array[float]): arreglo de parámetros
        x (array[float]): input
        y (array[float]): etiquetas

    Returns:
        float: fidelity entre la salida del circuito y la etiqueta
    """
    x1=x[:3]
    x2=x[3:6]
    x3=x[6:9]
    for p in params:
        qml.Rot(*x1, wires=0)
        qml.Rot(*x2, wires=0)
        qml.Rot(*x3, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))

def cost(params, x, y, state_labels=None):
    """Función de costo

    Args:
        params (array[float]): arreglo de parámetros
        x (array[float]): 2-d arreglo de inputs
        y (array[float]): 1-d arreglo de etiquetas
        state_labels (array[float]): arreglo de etiquetas como vectores de estado

    Returns:
        float: costo promedio sobre el batch
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x[i], dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

Testeo y batches

In [3]:
def test(params, x, y, state_labels=None):
    """
    Testea en un conjunto de datos dado los parámetros del circuito

    Args:
        params (array[float]): arreglo de parametros
        x (array[float]): 2-d arreglo de input vectors
        y (array[float]): 1-d arreglo de targets
        state_labels (array[float]): 1-d arreglo de representaciones de estado para las etiquetas
    Returns:
        predicted (array([int]): labels predichas para los datos de test
        output_states (array[float]): estados cuanticos de salida del circuito
    """
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = [fidel_function(dm) for dm in dm_labels]
        best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    """Precisión de la clasificación 

    Args:
        y_true (array[float]): 1-d arreglo de targets
        y_predicted (array[float]): 1-d arreglo de predicciones
        state_labels (array[float]): 1-d arreglo de representaciones de estado para las etiquetas

    Returns:
        score (float): fraccion de muestras clasificadas correctamente
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    """
    generador de batches

    Args:
        inputs (array[float]): data de entrada
        targets (array[float]): targets

    Returns:
        inputs (array[float]): un batch de inputs de longitud `batch_size`
        targets (array[float]): un batch de targets de longitud `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

Entrenamiento del circuito clasificador

In [4]:
#Datos de entrenamiento y test
import pandas as pd
from sklearn.model_selection import train_test_split

data=pd.read_csv('Preprocesamiento\sample1.csv')
Xdata=data[['r1','phi1','z1','r2','phi2','z2','r3','phi3','z3']].values
Ydata=(data['label'].values).astype(int)

#Division para 1 ejemplo:
X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size=0.1, random_state=42)
X_train = np.array(X_train, requires_grad=False)
X_test = np.array(X_test, requires_grad=False)
y_train = np.array(y_train, requires_grad=False)
y_test = np.array(y_test, requires_grad=False)

Ojo: entrenamiento con batches (quiza despues hagamos uno full batch como en el paper)

In [ ]:

# Entrenamiento del circuito clasificador
num_layers = 6
learning_rate = 0.01
epochs = 10
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)


# Inicializacion con parametros aleatorios
params = np.random.uniform(size=(num_layers, 3), requires_grad=True)
predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
accuracy_train = accuracy_score(y_train, predicted_train)

predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
accuracy_test = accuracy_score(y_test, predicted_test)

# guardamos las predicciones con pesos aleatorios para comparación
initial_predictions = predicted_test

loss = cost(params, X_test, y_test, state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)




Epoch:  0 | Cost: 0.342704 | Train accuracy: 0.481944 | Test Accuracy: 0.510000


In [10]:
import time
for it in range(epochs):
    inicio=time.time()
    for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
        params, _, _, _ = opt.step(cost, params, Xbatch, ybatch, state_labels)

    predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)
    loss = cost(params, X_train, y_train, state_labels)

    predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    fin=time.time()
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )
    print(f"\n {fin - inicio:.6f} segundos")

Epoch:  1 | Loss: 0.313629 | Train accuracy: 0.518056 | Test accuracy: 0.525000

 57.207161 segundos
Epoch:  2 | Loss: 0.303579 | Train accuracy: 0.523056 | Test accuracy: 0.495000

 58.957557 segundos
Epoch:  3 | Loss: 0.300164 | Train accuracy: 0.522222 | Test accuracy: 0.502500

 56.451406 segundos
Epoch:  4 | Loss: 0.296923 | Train accuracy: 0.522222 | Test accuracy: 0.502500

 58.106080 segundos
Epoch:  5 | Loss: 0.294101 | Train accuracy: 0.518889 | Test accuracy: 0.525000

 61.775801 segundos
Epoch:  6 | Loss: 0.292472 | Train accuracy: 0.523611 | Test accuracy: 0.535000

 60.440146 segundos
Epoch:  7 | Loss: 0.291466 | Train accuracy: 0.522778 | Test accuracy: 0.542500

 57.700117 segundos
Epoch:  8 | Loss: 0.290587 | Train accuracy: 0.525278 | Test accuracy: 0.542500

 55.548178 segundos
Epoch:  9 | Loss: 0.289627 | Train accuracy: 0.521389 | Test accuracy: 0.542500

 55.200867 segundos
Epoch: 10 | Loss: 0.288546 | Train accuracy: 0.519167 | Test accuracy: 0.532500

 54.471531

In [16]:
print(
    "Cost: {:3f} | Train accuracy {:3f} | Test Accuracy : {:3f}".format(
        loss, accuracy_train, accuracy_test
    )
)

print("Learned weights")
learned_params = params
for i in range(num_layers):
    print("Layer {}: {}".format(i, params[i]))

Cost: 0.133129 | Train accuracy 0.800000 | Test Accuracy : 0.640000
Learned weights
Layer 0: [0.04437666 0.85219353 0.03542237]
Layer 1: [0.32905137 0.93158949 0.73521265]
Layer 2: [0.28139096 0.25264909 0.00508214]


### II) 1-n EQK 

Embedding con parámetros aprendidos del QNN

In [5]:
def circuito_embedding(params, x, n):
    for p in params:
        #Rotaciones
        for i in range(n):
            qml.Rot(*x, wires=i)
            qml.Rot(*p, wires=i)
        
        #cascada de E (cnots)
        for i in range(n-1): qml.CNOT(wires=[i,i+1]) 



#adjunto

adjunto_empedding=qml.adjoint(circuito_embedding)

Creación circuito kernel con n=3

In [6]:
n=3
dev3 = qml.device("default.qubit", wires=n)
wires = dev3.wires.tolist()

@qml.qnode(dev3)
def circuito_kernel(x1,x2, params,n):
    circuito_embedding(params, x1, n)
    adjunto_empedding(params, x2, n)
    return qml.probs(wires=wires)

Valor de kernel

In [9]:
def kernel(x1, x2, params,n):
    return circuito_kernel(x1, x2, params,n)[0]

In [ ]:
#Probando kernel entre 2 puntos
kernel(X_train[0], X_train[1], learned_params, n)

tensor(0.28156454, requires_grad=True)

Construyendo matriz kernel

In [12]:
init_kernel = lambda x1, x2: kernel(x1, x2,learned_params, n)
#K_matrix = qml.kernels.square_kernel_matrix(X_train, init_kernel,assume_normalized_kernel=True)
#K_matrix.size

#### Usando matriz NQK kernel en SVC

In [2]:
from sklearn.svm import SVC
svm = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X_train,y_train) 

NameError: name 'X_train' is not defined

Accuracy medida

In [ ]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

nqk_accuracy = accuracy(svm, X_test, y_test)

Comparación de performances

In [ ]:
print("Test accuracy de QNN: ", accuracy_test)
print(f"Test accuracy de 1 to {n} NQK : {nqk_accuracy}")

Test accuracy de QNN:  0.64
Test accuracy de 1 to 3 NQK : 0.76


Se espera normalmente que el NQK tenga mayor o igual precision que el QNN. La precision está baja porque se usaron muy pocos datos de entrenamiento (deberian ser del orden de 1000) porque construir la matriz Kernel crece con O(n^2) y demora mucho en mi laptop para valores altos, quiza deberia probar en alguna pc de la facu.